<a href="https://colab.research.google.com/github/amandatz/NMF/blob/main/NMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NMF

In [1]:
using LinearAlgebra, Random, Plots, Statistics, Printf
using Base.Threads

In [2]:
using Random
Random.seed!(123)

TaskLocalRNG()

## Auxiliares

In [3]:
max_iter_fixed = 100000
tol_fixo = 1e-4
tol_fixo_sub = tol_fixo*0.01

1.0000000000000002e-6

In [4]:
function generate_matrix(m, n; type=:uniform)
    Q1 = qr(randn(m, m)).Q
    Q2 = qr(randn(n, n)).Q
    d = zeros(min(m, n))
    kappa = 1e8

    if type == :uniform
        d .= rand(length(d))

    elseif type == :decaying
        d .= LinRange(1.0, 0.1, length(d))

    elseif type == :equal
        d .= ones(length(d))

    elseif type == :ill_conditioned
        d .= [1 / kappa^((i - 1) / (length(d) - 1)) for i in 1:length(d)]

    else
        error("Tipo inválido")
    end

    X = abs.(Q1[:, 1:length(d)] * Diagonal(d) * Q2'[1:length(d), :])
    return X
end

function generate_X_WH(m, n, r; type=:uniform)
    W = generate_matrix(m, r; type=type)
    H = generate_matrix(r, n; type=type)
    X = W * H
    return X, W, H
end


generate_X_WH (generic function with 1 method)

In [5]:
function lipschitz_step(A; eps = 1e-8)
    L = opnorm(A) + eps
    return 1 / L
end

function rule_lipschitz_W(W, H, GW, iter, alpha_prev)
    return lipschitz_step(H * H')
end

function rule_lipschitz_H(W, H, GH, iter, alpha_prev)
    return lipschitz_step(W' * W)
end

rule_lipschitz_H (generic function with 1 method)

### Armijo

Busca Linear de Armijo com Backtracking

In [6]:
function armijo_step(
    F_func, X, G;
    alpha_init = 1e-3, beta = 0.5, c = 1e-4,
    max_iter = 20, proj = identity
)
    alpha = alpha_init
    F = F_func(X)
    normG2 = norm(G)^2
    for k = 1:max_iter
        X_new = proj(X .- alpha .* G)
        F_new = F_func(X_new)
        if F_new <= F - c * alpha * normG2
            return alpha
        end
        alpha *= beta
    end
    return alpha
end

function rule_armijo_W(W, H, GW, iter, alpha_prev)
    F_func = Wt -> 0.5 * norm(X - Wt * H)^2
    return armijo_step(F_func, W, GW; alpha_init=alpha_prev, proj=x->max.(x,0.0))
end

function rule_armijo_H(W, H, GH, iter, alpha_prev)
    F_func = Ht -> 0.5 * norm(X - W * Ht)^2
    return armijo_step(F_func, H, GH; alpha_init=alpha_prev, proj=x->max.(x,0.0))
end

rule_armijo_H (generic function with 1 method)

In [7]:
function rule_hybrid_H(W, H, GH, iter, alpha_prev)
    alpha_lip = 1 / (opnorm(W' * W) + 1e-8)
    F_func = Ht -> 0.5 * norm(X - W * Ht)^2
    return armijo_step(F_func, H, GH;
        alpha_init = alpha_lip,
        proj = x -> max.(x, 0.0)
    )
end

function rule_hybrid_W(W, H, GW, iter, alpha_prev)
    alpha_lip = 1 / (opnorm(H * H') + 1e-8)
    F_func = Wt -> 0.5 * norm(Wt * H - X)^2
    return armijo_step(F_func, W, GW;
        alpha_init = alpha_lip,
        proj = x -> max.(x, 0.0)
    )
end

function rule_hybrid_H(W, H, GH, iter, alpha_prev)
    alpha_lip = 1 / (opnorm(W' * W) + 1e-8)
    F_func = Ht -> 0.5 * norm(W * Ht - X)^2
    return armijo_step(F_func, H, GH;
        alpha_init = alpha_lip,
        proj = x -> max.(x, 0.0)
    )
end


rule_hybrid_H (generic function with 1 method)

### Barzilai–Borwein

$$
\alpha_k = \frac{s_k^T s_k}{s_k^T y_k}  
$$
com
$$
s_k = x_k - x_{k-1} \\
y_k = \nabla f(x_k) - \nabla f(x_{k-1})
$$

In [8]:
# ==========================================================
# Regra passo espectral (Barzilai–Borwein)
# ==========================================================
function make_rule_spectral_W()
    prev_W = nothing
    prev_G = nothing

    return (W, H, G, iter, alpha_prev) -> begin
        alpha = alpha_prev
        if prev_W !== nothing && prev_G !== nothing
            s = W .- prev_W
            y = G .- prev_G
            den = sum(s .* y)
            if den > 0 && isfinite(den)
                alpha = clamp(sum(s .* s) / den, 1e-12, 1e12)
            end
        end
        prev_W, prev_G = copy(W), copy(G)
        return alpha
    end
end


function make_rule_spectral_H()
    prev_H = nothing
    prev_G = nothing

    return (W, H, G, iter, alpha_prev) -> begin
        alpha = alpha_prev
        if prev_H !== nothing && prev_G !== nothing
            s = H .- prev_H
            y = G .- prev_G
            den = sum(s .* y)
            if den > 0 && isfinite(den)
                alpha = clamp(sum(s .* s) / den, 1e-12, 1e12)
            end
        end
        prev_H, prev_G = copy(H), copy(G)
        return alpha
    end
end


make_rule_spectral_H (generic function with 1 method)

## Outros métodos

### NMF Inversa

In [9]:
function nmf_inverse(X, r; alpha=0.1, max_iter=max_iter_fixed, tol=tol_fixo)
    m, n = size(X)
    W = max.(rand(m, r), 1e-4)
    H = max.(rand(r, n), 1e-4)
    errors = Float64[]

    t_start = time()
    iters = 0

    for iter = 1:max_iter
        W_old = copy(W); H_old = copy(H)

        A = H * H' + alpha*I(r)
        W = X * H' * inv(A)
        W .= max.(W, 0.0)
        H .= H .* ((W' * X) ./ max.(W' * W * H, 1e-8))
        H .= max.(H, 0.0)
        push!(errors, norm(X - W*H))

        if norm(W - W_old) / max(1, norm(W_old)) < tol &&
          norm(H - H_old) / max(1, norm(H_old)) < tol
            break
        end
    end

    t_end = time()
    total_time = t_end - t_start

    return W, H, errors, total_time, iters
end

nmf_inverse (generic function with 1 method)

### Gradiente Projetado (atualização simultânea)

$W^{t}$ e $H^{t}$ calculados simultaneamente. Espera-se terminar ambas $W^{t}$ e $H^{t}$ para calcular $W^{t+1}$ e $H^{t+1}$.

In [15]:
function nmf_pg_simultaneo(
    X, r;
    max_iter = max_iter_fixed,
    tol = tol_fixo,
    lambda = 0.0,
    alpha_rule_W = (W,H,G,i,a)->a,
    alpha_rule_H = (W,H,G,i,a)->a,
    alphaW_init = 1e-3,
    alphaH_init = 1e-3
)
    m, n = size(X)
    W = max.(rand(m, r), 1e-4)
    H = max.(rand(r, n), 1e-4)
    errors = Float64[]
    iters = 0

    alphaW = alphaW_init
    alphaH = alphaH_init

    t_start = time()

    for k = 1:max_iter
        W_old = copy(W)
        H_old = copy(H)

        #task_W = Threads.@spawn begin
            GW = W * (H * H') .- X * H' .+ lambda .* W
            alphaW = alpha_rule_W(W, H, GW, k, alphaW)
            W_new = max.(W .- alphaW .* GW, 0.0) #max.(W .- alphaW .* GW, 0.0), alphaW
        #end

        #task_H = Threads.@spawn begin
            GH = (W' * W) * H .- W' * X .+ lambda .* H
            alphaH = alpha_rule_H(W, H, GH, k, alphaH)
            H_new = max.(H .- alphaH .* GH, 0.0) #max.(H .- alphaH .* GH, 0.0), alphaH
        #end

        #(W_new, alphaW) = fetch(task_W)
        #(H_new, alphaH) = fetch(task_H)

        W .= W_new
        H .= H_new

        push!(errors, norm(X - W * H))
        iters = k

        # Critério de parada
        if norm(W - W_old) / max(1, norm(W_old)) < tol &&
           norm(H - H_old) / max(1, norm(H_old)) < tol
            break
        end
    end

    total_time = time() - t_start
    return W, H, errors, total_time, iters
end


nmf_pg_simultaneo (generic function with 1 method)

## NMF Multiplicativo (Lee & Seung)

In [10]:
function nmf_multiplicative(X, r; max_iter=max_iter_fixed, tol=tol_fixo)
    m, n = size(X)
    W = max.(rand(m, r), 1e-4)
    H = max.(rand(r, n), 1e-4)
    errors = Float64[]

    t_start = time()
    iters = 0

    for iter = 1:max_iter
        W_old = copy(W)
        H_old = copy(H)

        H .= H .* ((W' * X) ./ max.(W' * W * H, 1e-8))
        H .= max.(H, 0.0)
        W .= W .* ((X * H') ./ max.(W * (H * H'), 1e-8))
        W .= max.(W, 0.0)

        push!(errors, norm(X - W * H))
        iters = iter

        if norm(W - W_old) / max(1, norm(W_old)) < tol &&
           norm(H - H_old) / max(1, norm(H_old)) < tol
            break
        end
    end

    t_end = time()
    total_time = t_end - t_start
    return W, H, errors, total_time, iters
end

nmf_multiplicative (generic function with 1 method)

## Minimização Alternada com Gradiente Projetado (Lin)

## Gradiente Projetado (gradiente do subproblema)

In [11]:
function line_search_segment!(
    A_old, A_new, G, fixed_mat, f;
    beta = 0.5,
    sigma = 0.1,
    theta_min = 1e-12,
    monotone = true,
    f_hist = nothing,
    M_hist = 5 # número da memória
)
    # direcao
    dW = A_new .- A_old
    inner = sum(G .* dW)

    # valor no ponto antigo
    f_old = f(A_old, fixed_mat)

    # se não-monotono, calcula máximo da janela
    if monotone
        f_ref = f_old
    else
        # inicializa histórico se necessário
        if f_hist === nothing || isempty(f_hist)
            f_hist = [f_old]
        end

        # referencia = maior valor recente
        f_ref = maximum(f_hist)
    end

    # começa com passo cheio
    theta = 1.0
    f_new = f(A_new, fixed_mat)

    # condicao geral
    while f_new > f_ref - sigma * theta * inner
        theta *= beta
        if theta < theta_min
            break
        end
        A_new .= A_old .+ theta .* dW
        f_new = f(A_new, fixed_mat)
    end

    # se for não-monotono, atualiza histórico
    if !monotone
        push!(f_hist, f_new)
        if length(f_hist) > M_hist
            popfirst!(f_hist)  # mantém último M_hist valores
        end
    end

    return A_new, theta, f_hist
end

line_search_segment! (generic function with 1 method)

In [12]:
function projected_gradient_W(X, H, W0;
    alpha_init = 1e-3,
    lambda = 0.0,
    tol = tol_fixo_sub,
    max_iter = max_iter_fixed,
    monotone = true,
    alpha_rule_W = (W, H, G, iter, alpha_prev) -> alpha_prev
)
    W = copy(W0)
    alpha = alpha_init
    f_W(Wt,Ht) = 0.5 * norm(X - Wt*Ht)^2 + (lambda/2)*norm(Wt)^2
    f_hist_W = nothing

    for iter = 1:max_iter
        W_old = copy(W)

        GW = W * (H * H') .- X * H' .+ lambda .* W
        alpha = alpha_rule_W(W, H, GW, iter, alpha)
        W .= max.(W .- alpha .* GW, 0.0)

        W_new, theta, new_f_hist_W = line_search_segment!(
            W_old, W, GW, H, f_W;
            monotone = monotone,
            f_hist = f_hist_W,
            M_hist = 5
        )
        W .= W_new
        f_hist_W = new_f_hist_W

        if norm(W - W_old) / max(1.0, norm(W_old)) < tol
            return W, iter
        end
    end

    return W, max_iter
end


# ==========================================================
# Subproblema para H
# ==========================================================
function projected_gradient_H(X, W, H0;
    alpha_init = 1e-3,
    lambda = 0.0,
    tol = tol_fixo_sub,
    max_iter = max_iter_fixed,
    monotone = true,
    alpha_rule_H = (W, H, G, iter, alpha_prev) -> alpha_prev
)
    H = copy(H0)
    alpha = alpha_init
    f_H(Ht, Wt) = 0.5 * norm(X - Wt * Ht)^2 + (lambda/2)*norm(Ht)^2
    f_hist_H = nothing

    for iter = 1:max_iter
        H_old = copy(H)

        GH = (W' * W) * H .- W' * X .+ lambda .* H
        alpha = alpha_rule_H(W, H, GH, iter, alpha)
        H .= max.(H .- alpha .* GH, 0.0)

        H_new, theta, new_f_hist_H = line_search_segment!(
            H_old, H, GH, W, f_H;
            monotone = monotone,
            f_hist = f_hist_H,
            M_hist = 5
        )
        H .= H_new
        f_hist_H = new_f_hist_H

        if norm(H - H_old) / max(1.0, norm(H_old)) < tol
            return H, iter
        end
    end

    return H, max_iter
end

projected_gradient_H (generic function with 1 method)

In [13]:
function nmf_gradient_projected(
    X, r;
    max_iter = max_iter_fixed,
    tol = tol_fixo,
    alpha_init = 1e-3,
    lambda = 0.0,
    sub_max_iter = max_iter_fixed,
    sub_tol = tol_fixo_sub,
    monotone = true,
    alpha_rule_W = (W, H, G, i, a) -> a,
    alpha_rule_H = (W, H, G, i, a) -> a
)
    m, n = size(X)
    W = max.(rand(m, r), 1e-4)
    H = max.(rand(r, n), 1e-4)
    errors = Float64[]

    t_start = time()
    total_sub_iters = 0

    for iter = 1:max_iter
        W_old = copy(W)
        H_old = copy(H)

        # --- Atualização de W ---
        W, iter_W = projected_gradient_W(X, H, W;
            alpha_init = alpha_init,
            lambda = lambda,
            tol = sub_tol,
            max_iter = sub_max_iter,
            monotone = monotone,
            alpha_rule_W = alpha_rule_W
        )
        total_sub_iters += iter_W

        # --- Atualização de H ---
        H, iter_H = projected_gradient_H(X, W, H;
            alpha_init = alpha_init,
            lambda = lambda,
            tol = sub_tol,
            max_iter = sub_max_iter,
            monotone = monotone,
            alpha_rule_H = alpha_rule_H
        )
        total_sub_iters += iter_H

        push!(errors, norm(X - W * H))

        deltaW = norm(W - W_old) / max(1.0, norm(W_old))
        deltaH = norm(H - H_old) / max(1.0, norm(H_old))
        if deltaW < tol && deltaH < tol
            break
        end
    end

    total_time = time() - t_start
    return W, H, errors, total_time, total_sub_iters
end

nmf_gradient_projected (generic function with 1 method)

In [14]:
#function nmf_gradient_projected_(
#  X, r;
#  max_iter=max_iter_fixed,
#  tol=tol_fixo,
#  alpha_init=1e-3,
#  lambda=0.0,
#  alpha_rule_W = (W, H, GW, iter, alpha_prev) -> alpha_prev,
#  alpha_rule_H = (W, H, GH, iter, alpha_prev) -> alpha_prev)
#
#    m, n = size(X)
#    W = max.(rand(m, r), 1e-4)
#    H = max.(rand(r, n), 1e-4)
#    errors = Float64[]
#
#    alpha_W, alpha_H = alpha_init, alpha_init
#
#    t_start = time()
#    iters = 0
#
#    for iter = 1:max_iter
#        W_old = copy(W); H_old = copy(H)
#
#        # Atualização de W
#        GW = W * (H * H') .- X * H' .+ lambda .* W
#        alpha_W = alpha_rule_W(W, H, GW, iter, alpha_W)
#        W .= max.(W .- alpha_W .* GW, 0.0)
#
#        # Atualização de H
#        GH = (W' * W) * H .- W' * X .+ lambda .* H
#        alpha_H = alpha_rule_H(W, H, GH, iter, alpha_H)
#        H .= max.(H .- alpha_H .* GH, 0.0)
#
#        push!(errors, norm(X - W*H))
#        iters = iter
#
#        if norm(W - W_old) / max(1, norm(W_old)) < tol &&
#           norm(H - H_old) / max(1, norm(H_old)) < tol
#            break
#        end
#    end
#
#    t_end = time()
#    total_time = t_end - t_start
#    return W, H, errors, total_time, iters
#end


### Gradiente Projetado (atualização assíncrona)

$W^{t}$ e $H^{t}$ calculados simultaneamente. Não se espera terminar ambas $W^{t}$ e $H^{t}$ para calcular $W^{t+1}$ e $H^{t+1}$. Nesse caso, pode ocorrer de várias iterações $W^t, ..., W^{t+1}$ usarem o mesmo $H^t$.

In [16]:
function nmf_pg_async(
    X, r;
    max_iter = 5000,
    tol = 1e-4,
    lambda = 0.0,
    alpha_rule_W = (W,H,G,i,a)->a,
    alpha_rule_H = (W,H,G,i,a)->a,
    alphaW_init = 1e-3,
    alphaH_init = 1e-3
)
    m, n = size(X)
    W = max.(rand(m, r), 1e-4)
    H = max.(rand(r, n), 1e-4)
    errors = Float64[]

    lockW = ReentrantLock()
    lockH = ReentrantLock()

    stop_flag = Ref(false)
    iter = Ref(0)

    alphaW = alphaW_init
    alphaH = alphaH_init

    t_start = time()

    # Atualiza W
    taskW = Threads.@spawn begin
        while !stop_flag[] && iter[] < max_iter
            iter[] += 1

            lock(lockH)
            local H_local = copy(H)
            unlock(lockH)

            lock(lockW)
            GW = W * H_local * H_local' .- X * H_local' .+ lambda .* W
            alphaW = alpha_rule_W(W, H_local, GW, iter[], alphaW)
            W_new = max.(W .- alphaW .* GW, 0.0)
            ΔW = norm(W_new - W) / max(1, norm(W))
            W .= W_new
            unlock(lockW)

            if ΔW < tol
                stop_flag[] = true
            end
        end
    end

    # Atualiza H
    taskH = Threads.@spawn begin
        while !stop_flag[] && iter[] < max_iter
            lock(lockW)
            local W_local = copy(W)
            unlock(lockW)

            lock(lockH)
            GH = (W_local' * W_local) * H .- W_local' * X .+ lambda .* H
            alphaH = alpha_rule_H(W_local, H, GH, iter[], alphaH)
            H_new = max.(H .- alphaH .* GH, 0.0)
            ΔH = norm(H_new - H) / max(1, norm(H))
            H .= H_new
            unlock(lockH)

            if ΔH < tol
                stop_flag[] = true
            end
        end
    end

    # Monitora erro periodicamente
    while !stop_flag[] && iter[] < max_iter
        sleep(0.01)
        lock(lockW); lock(lockH)
        push!(errors, norm(X - W * H))
        unlock(lockH); unlock(lockW)
    end

    total_time = time() - t_start
    fetch(taskW); fetch(taskH)

    return W, H, errors, total_time, iter[]
end


nmf_pg_async (generic function with 1 method)

In [17]:
println("Threads disponíveis: ", Threads.nthreads())

X = rand(10, 10)
W, H, errors, t, iters = nmf_pg_async(X, 2;)
println("Erro final = $(errors[end])")
println("Tempo total = $(round(t, digits=3)) s")
println("Iterações ≈ $iters")

Threads disponíveis: 2
Erro final = 2.3924952525474725
Tempo total = 3.232 s
Iterações ≈ 962


## Testes

In [18]:
dims = [100, 1000]
r_values = [5, 10, 20]
types = [:equal, :ill_conditioned]

#dims = [1000]
#r_values = [5, 100, 500]
#types = [:equal, :ill_conditioned]

models = Dict{Symbol, Function}(
    #:inversa => nmf_inverse,

    # ------------
    :multiplicativo => nmf_multiplicative,

    # ------------
    #:gradiente_projetado => (X, r; kwargs...) -> nmf_gradient_projected( # passo fixo
    #    X, r; alpha_rule_W = (W,H,G,i,a)->a, alpha_rule_H = (W,H,G,i,a)->a, kwargs...
    #),

    #:gradiente_projetado_lipschitz => (X, r; kwargs...) -> nmf_gradient_projected( # Lipschitz
    #    X, r; alpha_rule_W = rule_lipschitz_W, alpha_rule_H = rule_lipschitz_H, kwargs...
    #),

    #:gradiente_projetado_armijo => (X, r; kwargs...) -> nmf_gradient_projected( # Armijo
    #    X, r; alpha_rule_W = rule_armijo_W, alpha_rule_H = rule_armijo_H, kwargs...
    #),

    :gradiente_projetado_spectral_non_monotone => (X, r; kwargs...) -> nmf_gradient_projected(
        X, r;
        alpha_rule_W = make_rule_spectral_W(),
        alpha_rule_H = make_rule_spectral_H(),
        monotone = false,
        kwargs...
    ),

    :gradiente_projetado_spectral_monotone => (X, r; kwargs...) -> nmf_gradient_projected(
        X, r;
        alpha_rule_W = make_rule_spectral_W(),
        alpha_rule_H = make_rule_spectral_H(),
        monotone = true,
        kwargs...
    ),
)

Dict{Symbol, Function} with 3 entries:
  :gradiente_projetado_spectral_monotone     => #79
  :multiplicativo                            => nmf_multiplicative
  :gradiente_projetado_spectral_non_monotone => #77

### Matrizes aleatórias $X, W, H$

In [ ]:
for n in dims
    for r in r_values
        if r > n
            continue
        end

        for t in types
            println("\n=== Dimension = $n | rank = $r | type = $t ===")
            X = generate_matrix(n,n, type=t)

            for (name, model) in models
                W, H, err, total_time, iters = model(X, r)

                erro_final = isempty(err) ? NaN : err[end]
                tempo = round(total_time, digits=4)

                println("$(string(name)) → Erro final: $erro_final | Iterações: $iters | Tempo: $(tempo)s")
            end
        end
    end
end


=== Dimension = 100 | rank = 5 | type = equal ===
gradiente_projetado_spectral_monotone → Erro final: 5.568861531982 | Iterações: 502 | Tempo: 4.1235s
multiplicativo → Erro final: 5.550016565382152 | Iterações: 1922 | Tempo: 0.2146s
gradiente_projetado_spectral_non_monotone → Erro final: 5.585246024075872 | Iterações: 296 | Tempo: 0.0578s

=== Dimension = 100 | rank = 5 | type = ill_conditioned ===
gradiente_projetado_spectral_monotone → Erro final: 0.8055070371867611 | Iterações: 160 | Tempo: 0.0304s
multiplicativo → Erro final: 0.7778718260148995 | Iterações: 1021 | Tempo: 0.112s
gradiente_projetado_spectral_non_monotone → Erro final: 0.7831729286922701 | Iterações: 307 | Tempo: 0.0543s

=== Dimension = 100 | rank = 10 | type = equal ===
gradiente_projetado_spectral_monotone → Erro final: 5.153741144712263 | Iterações: 921 | Tempo: 0.1817s
multiplicativo → Erro final: 5.134837090487826 | Iterações: 1759 | Tempo: 0.22s
gradiente_projetado_spectral_non_monotone → Erro final: 5.1713406

### Teste exato $X = WH$

In [ ]:
dims = [10, 100, 1000]
r_values = [10]
types = [:equal]

for n in dims
    for r in r_values
        if r > n
            continue
        end

        for t in types
            println("\n=== Dimension = $n | rank = $r | type = $t ===")
            X, W_true, H_true = generate_X_WH(n, n, r; type=t)

            for (name, model) in models
                W, H, err, total_time, iters = model(X, r)

                erro_final = isempty(err) ? NaN : err[end]
                tempo = round(total_time, digits=4)

                err_WH = norm(X - W * H) / norm(X)
                err_W  = norm(W - W_true) / norm(W_true)
                err_H  = norm(H - H_true) / norm(H_true)
                println("$(string(name)) → Erro(X): $(round(err_WH,digits=6)) | Erro(W): $(round(err_W,digits=6)) | Erro(H): $(round(err_H,digits=6)) | Iterações: $iters | Tempo: $(tempo)s")
            end
        end
    end
end

In [ ]:
num_trials = 10
dims = [200]
r = 10
type = :uniform

for n in dims
    println("\n===============================")
    println("=== Dimensão = $n | rank = $r | tipo = $type ===")
    println("===============================")

    # inicializa dicionários
    errors = Dict(model => [] for model in keys(models))
    times  = Dict(model => Float64[] for model in keys(models))
    iters  = Dict(model => Int[] for model in keys(models))

    # ----------------------------------------------------------
    # Loop principal
    # ----------------------------------------------------------
    for trial in 1:num_trials
        print("$trial, ")
        X = generate_matrix(n, n, type=type)

        for (name, model) in models
            t_start = time()
            _, _, err = model(X, r)
            t_end = time()

            push!(errors[name], err)
            push!(times[name], t_end - t_start)
            push!(iters[name], length(err))
        end
    end
    println("\n")

    # ----------------------------------------------------------
    # Estatísticas de erro final
    # ----------------------------------------------------------
    println("=== Estatísticas do erro final (n = $n) ===")
    for name in keys(models)
        final_errs = [isempty(e) ? NaN : e[end] for e in errors[name]]
        media = mean(skipmissing(final_errs))
        desvio = std(skipmissing(final_errs))
        @printf("%-35s → Média = %.6f | Std = %.6f\n", string(name), media, desvio)
    end

    # ----------------------------------------------------------
    # Estatísticas de tempo
    # ----------------------------------------------------------
    println("\n=== Estatísticas de tempo (n = $n) ===")
    for name in keys(models)
        media = mean(times[name])
        desvio = std(times[name])
        @printf("%-35s → Média = %.6fs | Std = %.6fs\n", string(name), media, desvio)
    end

    # ----------------------------------------------------------
    # Estatísticas de iterações
    # ----------------------------------------------------------
    println("\n=== Estatísticas do número de iterações (n = $n) ===")
    for name in keys(models)
        media = mean(iters[name])
        desvio = std(iters[name])
        @printf("%-35s → Média = %.2f | Std = %.2f\n", string(name), media, desvio)
    end
end


# NMF package

In [ ]:
## https://nimfa.biolab.si/nimfa.datasets.html

In [ ]:
using Pkg
Pkg.add(url="https://github.com/JuliaStats/NMF.jl")
using NMF


     Cloning git-repo `https://github.com/JuliaStats/NMF.jl`
    Updating git-repo `https://github.com/JuliaStats/NMF.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed NonNegLeastSquares ─ v0.4.1
   Installed RandomizedLinAlg ─── v0.1.0
    Updating `~/.julia/environments/v1.11/Project.toml`
  [6ef6ca0d] + NMF v1.0.3 `https://github.com/JuliaStats/NMF.jl#master`
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6ef6ca0d] + NMF v1.0.3 `https://github.com/JuliaStats/NMF.jl#master`
  [b7351bd1] + NonNegLeastSquares v0.4.1
  [0448d7d9] + RandomizedLinAlg v0.1.0
Precompiling project...
   4984.0 ms  ✓ NonNegLeastSquares
   3881.1 ms  ✓ RandomizedLinAlg
  42869.2 ms  ✓ NMF
  3 dependencies successfully precompiled in 55 seconds. 490 already precompiled.


http://mtm.ufsc.br/~douglas/2019.1/MTM5813/matlab/